##### Copyright 2020 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Point Clouds for tensorflow_graphics
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/schellmi42/tensorflow_graphics_point_clouds/blob/master/pylib/notebooks/Introduction.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/schellmi42/tensorflow_graphics_point_clouds/blob/master/pylib/notebooks/Introduction.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Initialization


### Clone repositories, and install requirements and custom_op package

In [ ]:
# Clone repositories
!rm -r tensorflow_graphics_point_clouds
!rm -r graphics
!git clone https://github.com/schellmi42/tensorflow_graphics_point_clouds
!git clone https://github.com/schellmi42/graphics

# install requirements and load tfg module 
!pip install -r graphics/requirements.txt

# install custom ops
!pip install tensorflow_graphics_point_clouds/custom_ops/pkg_builds/tf_2.2.0/*.whl


### Load modules

In [ ]:
import sys
# (this is equivalent to export PYTHONPATH='$HOME/graphics:/content/graphics:$PYTHONPATH', but adds path to running session)
sys.path.append("/content/graphics")

# load point cloud module 
# (this is equivalent to export PYTHONPATH='/content/tensorflow_graphics_point_clouds:$PYTHONPATH', but adds path to running session)
sys.path.append("/content/tensorflow_graphics_point_clouds")

Check if it loads without errors

In [ ]:
import tensorflow as tf
import tensorflow_graphics as tfg
import MCCNN2.pc as pc
import numpy as np

print('TensorFlow version: %s'%tf.__version__)
print('TensorFlow-Graphics version: %s'%tfg.__version__)
print('Point Cloud Module: ', pc)

## Example Code


### 2D square point clouds using segmentation IDs
Here we create a batch of point clouds with variable number of points per cloud from unordered points with an additional id tensor.

The `batch_ids` are the segmentation ids, which indicate which point belongs to which point cloud in the batch. For more information on segmentation IDs see: [tf.math#segmentation](https://www.tensorflow.org/api_docs/python/tf/math#Segmentation)

If the points are ordered by batch id, it is also possible to pass a `sizes` tensor, which has the size of each point cloud in it.

In [ ]:
import numpy as np


def square(num_samples, size=1):
  # 2D square in 3D for easier visualization
  points = np.random.rand(num_samples, 2)*2-1
  return points*size

num_samples=1000
batch_size = 10

# create numpy input data consisting of points and segmentation identifiers
points = square(num_samples)
batch_ids = np.random.randint(0, batch_size, num_samples)

# create tensorflow point cloud
point_cloud = pc.PointCloud(points, batch_ids, batch_size)

# print information
sizes = point_cloud.get_sizes()
print('%s point clouds of sizes:'%point_cloud._batch_size)
print(sizes.numpy())

Create a batch of point hierarchies using sequential poisson disk sampling with pooling radii 0.1, 0.4, 2.

In [ ]:
# numpy input parameters
sampling_radii = np.array([[0.1], [0.4], [2]])

# create tensorflow point hierarchy
point_hierarchy = pc.PointHierarchy(point_cloud,
                                    sampling_radii,
                                    'poisson_disk')

In [ ]:
# print information
num_levels = len(sampling_radii) + 1
print('%s point clouds of sizes:'%point_cloud._batch_size)
sizes = point_hierarchy.get_sizes()
for i in range(num_levels):
  print('level: ' + str(i))
  print(sizes[i].numpy())

assign a shape to the batch and look at the sizes again

In [ ]:
point_hierarchy.set_batch_shape([2, 5])
print('%s point clouds of sizes:'%point_cloud._batch_size)
sizes = point_hierarchy.get_sizes()
for i in range(num_levels):
  print('level: ' + str(i))
  print(sizes[i].numpy())

Visualize the levels of one example from the batch.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# which example from the batch to choose, can be 'int' or relative in [A1,...,An]
batch_id = [0,1]

curr_points = point_hierarchy.get_points(batch_id)

# plotting
plt.figure(figsize=[num_levels*5,5])
for i in range(num_levels):
  plt.subplot(1,num_levels,i+1)
  plt.plot(curr_points[i][:, 0],curr_points[i][:, 1],'bo')
  plt.axis([-1, 1, -1, 1])
  if i==0:
    plt.title('input point cloud')
  else:
    plt.title('poisson sampled points with radius %s'%sampling_radii[i - 1, 0])
    


### 3D point clouds from input files using arbitrary batch sizes with padding

Here we create point clouds from input files using a zero padded representation of shape `[A1, .., An, V, D]`.
Internally this is converted to a segmented representation.



 #### Loading from ASCII .txt files

In [ ]:
import MCCNN2.io as io

# SHREC15

#### get files ####
input_dir = 'tensorflow_graphics_point_clouds/MCCNN2/test_point_clouds/SHREC15/'
filenames = tf.io.gfile.listdir(input_dir)
batch_size = len(filenames)
print('### batch size ###'); print(batch_size)

for i in range(batch_size):
  filenames[i] = input_dir + filenames[i]

#### load points #####
batch_shape = [5,2]
print('### batch shape###'); print(batch_shape)
points, normals, sizes = io.load_batch_of_points(filenames, batch_shape = batch_shape)

print('### data shape ###'); print(points.shape)
print('### points per point cloud ###');print(sizes.numpy())

#### build point hierarchy #####
point_cloud = pc.PointCloud(points, sizes=sizes)

point_hierarchy = pc.PointHierarchy(point_cloud,
                                    [[0.05], [0.1]],
                                    'poisson_disk')

sizes = point_hierarchy.get_sizes()

print('### point per point cloud in hierarchy ###')
for level in range(len(sizes)):
  print('level %s'%level)
  print(sizes[level].numpy())

### extract points from last level in original batch shape ###
hierarchical_points = point_hierarchy.get_points()
out_points = hierarchical_points[-1]
print('### shape of points in last level ###'); print(out_points.shape)

#### Loading vertices from mesh files 


In [ ]:
# Thingi10k meshes

#### get files ####
input_dir = 'tensorflow_graphics_point_clouds/MCCNN2/test_point_clouds/meshes/'
filenames = tf.io.gfile.listdir(input_dir)
batch_size = len(filenames)
print('### batch size ###'); print(batch_size)

for i in range(batch_size):
  filenames[i] = input_dir+filenames[i]

#### load points ####
points, sizes = io.load_batch_of_meshes(filenames)

print('### data shape ###'); print(points.shape)
print('### points per point cloud ###');print(sizes.numpy())

#### build a point cloud object ####
point_cloud = pc.PointCloud(points, sizes=sizes)

print('### internal shape conversion ###')
print('Input    (padded):    %s elements'%len(tf.reshape(points, [-1, 3])))
print('Internal (segmented): %s elements'%len(point_cloud._points))

point_hierarchy = pc.PointHierarchy(point_cloud,
                                    [[0.05], [0.1]],
                                    'poisson_disk')

sizes = point_hierarchy.get_sizes()

print('### point per point cloud in hierarchy ###')
for level in range(len(sizes)):
  print('level %s'%level)
  print(sizes[level].numpy())

### Monte-Carlo Convolutions


Create convolutions for a point hierarchy with MLPs as kernel 



In [ ]:
import numpy as np
### create random input data
num_pts = 1000
point_dim = 3
feature_dim = 3
batch_size = 10

# create random points
points = np.random.rand(num_pts,point_dim)
batch_ids = np.random.randint(0,batch_size,num_pts)
batch_ids[:batch_size] = np.arange(0,batch_size) # ensure non-empty point clouds
# create random features
features = np.random.rand(num_pts,feature_dim)

# build initial point cloud
point_cloud = pc.PointCloud(points, batch_ids, batch_size)

# build point hierarchy
sample_radii = np.array([[0.1],[0.2],[2]])
point_hierarchy = pc.PointHierarchy(point_cloud,sample_radii)

### build model

# layer parameters
conv_radii = 2*sample_radii
feature_sizes = [8,16,32]
kernel_hidden_size = 8 # number of neurons in the hidden layer of the kernel MLP

### initialize layers
Conv1 = pc.layers.MCConv(feature_dim, feature_sizes[0], point_dim,kernel_hidden_size)
Conv2 = pc.layers.MCConv(feature_sizes[0],feature_sizes[1],point_dim,kernel_hidden_size)
Conv3 = pc.layers.MCConv(feature_sizes[1],feature_sizes[2],point_dim,kernel_hidden_size)

### call layers
conv1_result = Conv1(features,point_hierarchy[0], point_hierarchy[1],conv_radii[0])
conv2_result = Conv2(conv1_result,point_hierarchy[1], point_hierarchy[2],conv_radii[1])
conv3_result = Conv3(conv2_result,point_hierarchy[2], point_hierarchy[3],conv_radii[2], return_sorted=True)

### printing 
print('### point cloud sizes ###')
sizes = point_hierarchy.get_sizes()
for s in sizes:
  print(s.numpy())

print('\n### features dimensions flat ###')
print('Input: ');print(features.shape)
print('Conv1: ');print(conv1_result.shape)
print('Conv2: ');print(conv2_result.shape)
print('Conv3: ');print(conv3_result.shape)

# again in padded format
point_hierarchy.set_batch_shape([5,2])

unflatten = point_hierarchy[0].get_unflatten()
features_padded = unflatten(features)
### call layers
conv1_result_padded = Conv1(features_padded, point_hierarchy[0], point_hierarchy[1],conv_radii[0], return_padded=True)
conv2_result_padded = Conv2(conv1_result_padded, point_hierarchy[1], point_hierarchy[2],conv_radii[1], return_padded=True)
conv3_result_padded = Conv3(conv2_result_padded, point_hierarchy[2], point_hierarchy[3],conv_radii[2], return_padded=True)
print('\n### feature dimensions padded ###')
print('Input: ');print(features_padded.shape)
print('Conv1: ');print(conv1_result_padded.shape)
print('Conv2: ');print(conv2_result_padded.shape)
print('Conv3: ');print(conv3_result_padded.shape)

### ResNet blocks and pooling layers

In [ ]:

### create random input data
num_pts = 1000
point_dim = 3
feature_dim = 3
batch_size = 10

# create random points
points = np.random.rand(num_pts,point_dim)
batch_ids = np.random.randint(0,batch_size,num_pts)
batch_ids[:batch_size] = np.arange(0,batch_size) # ensure non-empty point clouds
# create random features
features = np.random.rand(num_pts,feature_dim)

# build initial point cloud
point_cloud = pc.PointCloud(points, batch_ids, batch_size)

# build point hierarchy
sample_radii = np.array([[0.1]])
point_hierarchy = pc.PointHierarchy(point_cloud,sample_radii)

### build model

# layer parameters
conv_radii = np.array([0.2, 0.5])
feature_sizes = [8]
kernel_hidden_size = 8 # number of neurons in the hidden layer of the kernel MLP
num_resnet_blocks =4 # number of ResNet blocks, each block has 2 layers

### initialize layers
Conv = pc.layers.MCConv(feature_dim, feature_sizes[0], point_dim, kernel_hidden_size)
ResNet = pc.layers.MCResNet(feature_sizes[0], num_resnet_blocks, point_dim, kernel_hidden_size)
Pool = pc.layers.GlobalMaxPooling()

### call layers
conv_result = Conv(features, point_hierarchy[0], point_hierarchy[1], conv_radii[0])
resnet_result = ResNet(conv_result, point_hierarchy[1], conv_radii[1], training=True)
pool_result = Pool(resnet_result, point_hierarchy[1])

### printing 
print('### point cloud sizes ###')
sizes = point_hierarchy.get_sizes()
for s in sizes:
  print(s.numpy())

print('\n### features dimensions flat ###')
print('Input: ');print(features.shape)
print('Conv: ');print(conv_result.shape)
print('ResNet: ');print(resnet_result.shape)
print('GlobalPool: ');print(pool_result.shape)

# again in padded format

point_hierarchy.set_batch_shape([5,2])
unflatten = point_hierarchy[0].get_unflatten()
features_padded = unflatten(features)
### call layers
conv_result_padded = Conv1(features_padded,point_hierarchy[0], point_hierarchy[1],conv_radii[0], return_padded=True)
resnet_result_padded = ResNet(conv_result_padded, point_hierarchy[1], conv_radii[1], training=True, return_padded=True)
pool_result_padded = Pool(resnet_result_padded, point_hierarchy[1], return_padded=True)
print('\n### feature dimensions padded ###')
print('Input: ');print(features_padded.shape)
print('Conv: ');print(conv_result_padded.shape)
print('ResNet: ');print(resnet_result_padded.shape)
print('GlobalPool: ');print(pool_result_padded.shape)

### Optimizations

#### Reuse precomputed data structure

In [ ]:
import tensorflow as tf 
import MCCNN2.pc as pc
import numpy as np
# method to create a random point cloud
from MCCNN2.pc.tests.utils import _create_random_point_cloud_segmented

# module for timing
import time


class conv_model():
  """ A Monte-Carlo convolutional neural network (MCCNN) without downsampling,
  just repeated convolutions
  """
  def __init__(self, num_features, depth):
    """ Constructor.

    Args:
      num_features: `int`, the dimensionality of the features.
      depth: `int` the number of layers.
    """
    self.depth = depth
    self.layers = [pc.layers.MCConv(num_features_in=num_features,
                                    num_features_out=num_features,
                                    num_dims=3,
                                    size_hidden=8)
                   for i in range(depth)]
  
  def __call__(self, point_cloud, features, radius, neighborhood=None):
    """ Evaluates the network.

    Args:
      point_cloud: A `PointCloud` instance, with N points.
      features: A `float` tensor of shape [N,C], where C is `num_features`
        from constructor.
      radius: An `float`, the radius of the convolution.
      neighborhood: A `Neighborhood` instance for `point_cloud` with cell_size radius, 
        if `None` a neighborhood is computed internally.

      Returns:
        A tensor of shape [N,C], the result of the convolutions.

    """
    for layer in self.layers:
      features = layer(features, point_cloud, point_cloud, radius=radius, neighborhood=neighborhood)
    return features


# hyper parameters
def time_networks(num_points, radius):
  if num_points >= 1e6:
    print('\n### number of points %sM ###'%(num_points//1000000))
  else:
    print('\n### number of points %sk ###'%(num_points//1000))
  
  batch_size = 32
  num_features = 8
  depth = 10

  # initialize the network 
  MCCNN = conv_model(num_features, depth)

  # create random input
  points, batch_ids = _create_random_point_cloud_segmented(batch_size, num_points, dimension=3)
  features = np.random.rand(num_points, num_features)
  point_cloud = pc.PointCloud(points, batch_ids)

  #### execute without precomupted neighborhood
  t1 = time.time()
  conv_result_1 = MCCNN(point_cloud, features, radius)
  t2 = time.time()

  print('recompute neighbors: %.3fs'%(t2-t1))

  # precompute neighborhood
  cell_sizes = [radius, radius, radius]
  grid = pc.Grid(point_cloud, cell_sizes)
  neighborhood = pc.Neighborhood(grid, cell_sizes)

  # execute with precomputed neighborhood
  t1 = time.time()
  conv_result_2 = MCCNN(point_cloud, features, radius, neighborhood)
  t2 = time.time()
  print('reuse neighbors:     %.3fs'%(t2-t1))

time_networks(10000, 0.1)
time_networks(100000, 0.1)
time_networks(1000000, 0.01)




#### Transpose Neighborhoods

In [ ]:
from MCCNN2.pc.tests.utils import _create_random_point_cloud_segmented
from MCCNN2.pc import PointCloud, Grid, Neighborhood
import numpy as np
import time

def time_transpose_neighborhood(num_points, batch_size=8, num_reps=10):
  if num_points >= 1e6:
    print('\n### number of points %sM ###'%(num_points//1000000))
  else:
    print('\n### number of points %sk ###'%(num_points//1000))
  num_poins = num_points
  num_centers = max(num_points // 100, batch_size)
  radius = 0.1
  
  nbs = []
  point_clouds = []
  point_clouds_centers = []
  for i in range(num_reps):
    points, batch_ids = _create_random_point_cloud_segmented(batch_size, num_points)
    point_cloud = PointCloud(points, batch_ids)
    points_centers, batch_ids_centers = _create_random_point_cloud_segmented(batch_size, num_centers)
    point_cloud_centers = PointCloud(points_centers, batch_ids_centers)
    grid = Grid(point_cloud, radius)

    point_clouds.append(point_cloud)
    point_clouds_centers.append(point_cloud_centers)
    nbs.append(Neighborhood(grid, radius, point_cloud_centers))

  t1 = time.time()
  for i in range(num_reps):
    grid_centers = Grid(point_clouds_centers[i], radius)
    nb_t1 = Neighborhood(grid_centers, radius, point_clouds[i])
  t2 = time.time()
  t = 1000*(t2-t1)/num_reps
  print('compute new neighborhood:        %.1fms'%(t))

  t3 = time.time()
  for i in range(num_reps):
    nb_t2 = nbs[i].transpose()
  t4 = time.time()
  t = 1000*(t4-t3)/num_reps
  print('transpose existing neighborhood: %.1fms'%(t))

time_transpose_neighborhood(1000, num_reps=100)
time_transpose_neighborhood(10000, num_reps=100)
time_transpose_neighborhood(100000, num_reps=10)
time_transpose_neighborhood(1000000, num_reps=10)

#### 1x1 Convolutions

In [ ]:
import tensorflow as tf
import MCCNN2.pc as pc
import numpy as np
# timing utilities
import time
# function to create a random point cloud
from MCCNN2.pc.tests.utils import _create_random_point_cloud_segmented
# custom gpu and tensorflow implementation of compute_pdf
from MCCNN2.pc.layers import Conv1x1, MCConv


def time_1x1_conv(num_points,
                  batch_size,
                  num_feat,
                  radius,
                  hidden_size,
                  dimension):
  # create random point cloud
  points, batch_ids = _create_random_point_cloud_segmented(
      batch_size, num_points, dimension=dimension)
  point_cloud = pc.PointCloud(points, batch_ids)
  features = np.random.rand(num_points, num_feat[0])
  # build conv_layers
  conv1x1 = Conv1x1(num_feat[0], num_feat[1])
  mcconv = MCConv(num_feat[0], num_feat[1], dimension, hidden_size)
  # build 1x1 neighborhood
  cell_sizes = np.float32(np.repeat(radius, dimension))
  grid = pc.Grid(point_cloud, cell_sizes)
  neighborhood = pc.Neighborhood(grid, radius)
  # is1x1 = neighborhood._neighbors.shape[0] == num_points
  # print('\nMCConv is 1x1: ', is1x1)
    
  # compute conv
  t1 = time.time()
  neighbors  = tf.stack((tf.range(0, num_points), tf.range(0, num_points)), axis=1)
  _ = mcconv(features, point_cloud, point_cloud, radius, neighborhood)
  t2 = time.time()
  _ = conv1x1(features, point_cloud)
  t3 = time.time()

  # printing
  if num_points >= 1e6: 
    print('num points: %sM, feature dim: %s -> %s'%(num_points//int(1e6), num_feat[0], num_feat[1]))
  elif num_points >= 1e3:
    print('num points: %sk, feature dim: %s -> %s'%(num_points//int(1e3), num_feat[0], num_feat[1]))
  else:
    print('\nnum points: %s'%num_points)
  
  num_p_mc = (1 + num_feat[0]) * hidden_size + hidden_size * num_feat[0] * num_feat[1]
  num_p_1x1 = num_feat[0] * num_feat[1]
  print('num params: %10d, time MCCNN:       %.4fs'%(num_p_mc, t2-t1))
  print('num params: %10d, time Conv1x1:     %.4fs\n'%(num_p_1x1, t3-t2))



time_1x1_conv(10000, 32, [1, 1], 1e-3, 8, 3)
time_1x1_conv(10000, 32, [64, 64], 1e-3, 8, 3)
time_1x1_conv(10000, 32, [1, 2048], 1e-3, 8, 3)
time_1x1_conv(10000, 32, [2048, 1], 1e-3, 8, 3)
time_1x1_conv(10000, 32, [2048, 2048], 1e-3, 8, 3)